In [1]:
%pip install pandas




Note: you may need to restart the kernel to use updated packages.


In [63]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

options = Options()
# # options.add_argument('--headless')  # Uncomment for headless mode
driver = webdriver.Chrome(service=Service(), options=options)

# Your genre URLs
genre_urls = {
    'Romance': 'https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=romance',
    'Action': 'https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=action',
    'Music': 'https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=music',
    'Reality': 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=2024-01-01,2024-12-31&genres=reality-tv',
    'Talk show': 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=2024-01-01,2024-12-31&genres=talk-show'
}

# --- FUNCTION TO CLICK "LOAD MORE" ---
def click_load_more():
    try:
        load_more_button = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]//button//span[contains(text(),"more")]'
        )
        ActionChains(driver).move_to_element(load_more_button).perform()
        load_more_button.click()
        time.sleep(3)
        return True
    except Exception as e:
        print("No more 'Load More' button or error:", e)
        return False

# --- MAIN SCRAPER LOOP ---
for genre, url in genre_urls.items():
    print(f"Scraping {genre}...")
    driver.get(url)
    time.sleep(3)

    # Keep clicking "Load More" if available
    while click_load_more():
        print("Clicked 'Load More'...")

    titles = driver.find_elements(By.CSS_SELECTOR, 'h3.ipc-title__text')
    ratings = driver.find_elements(By.CSS_SELECTOR, 'span.ipc-rating-star--rating')
    votes = driver.find_elements(By.CSS_SELECTOR, 'span.ipc-rating-star--voteCount')

    metadata_blocks = driver.find_elements(By.XPATH, '//div[contains(@class, "dli-title-metadata")]')
    durations = []

    for block in metadata_blocks:
        spans = block.find_elements(By.TAG_NAME, 'span')
        found_duration = ""
        for span in spans:
            text = span.text.strip()
            if "h" in text or "m" in text:
                found_duration = text
                break
        durations.append(found_duration)

    data = []
    for i in range(min(len(titles), len(ratings), len(votes), len(durations))):
        data.append({
            'Movie Name': titles[i].text,
            'Genre': genre,
            'Rating': ratings[i].text,
            'Votes': votes[i].text,
            'Duration': durations[i]
        })

    if data:
        df = pd.DataFrame(data)
        file_name = genre.lower().replace(" ", "_") + "_movies.csv"
        df.to_csv(file_name, index=False)
        print(f"Saved: {file_name}")
    else:
        print(f"No data found for {genre}.")

driver.quit()


Scraping Romance...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
Clicked 'Load More'...
No more 'Load More' button or error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]//button//span[contains(text(),"more")]"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff68b86e925+77845]
	GetHandleV

In [64]:


merge = pd.concat([
    pd.read_csv("romance_movies.csv"),
    pd.read_csv("action_movies.csv"),
    pd.read_csv("music_movies.csv"),
    pd.read_csv("reality_movies.csv"),
    pd.read_csv("talk_show_movies.csv")
], ignore_index=True)

# Optional: save or view
merge.to_csv("all_genre_movies.csv", index=False)
print(merge.head())

           Movie Name    Genre  Rating    Votes Duration
0           1. Wicked  Romance     7.4   (174K)   2h 40m
1            2. Anora  Romance     7.5   (211K)   2h 19m
2  3. We Live in Time  Romance     7.0    (59K)   1h 48m
3         4. Babygirl  Romance     5.8    (65K)   1h 54m
4     5. The Fall Guy  Romance     6.8   (229K)    2h 6m


In [ ]:
import sqlite3
import csv

# Step 1: Connect to the database
con = sqlite3.connect("films.db")
cur = con.cursor()

# Step 2: Create the table (with correct names — no spaces)
cur.execute('''
CREATE TABLE IF NOT EXISTS film (
    MovieName TEXT,
    Genre TEXT,
    Rating REAL,
    Votes INTEGER,
    Duration TEXT
)
''')

# Step 3: Open CSV and insert rows
with open("all_genre_movies.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row

    for row in reader:
        # row = ['Anora', 'Romance', '7.5', '170000', '2h 40m']
        cur.execute("""
            INSERT INTO film (MovieName, Genre, Rating, Votes, Duration)
            VALUES (?, ?, ?, ?, ?)
        """, row)

# Step 4: Commit and close
con.commit()
con.close()


In [ ]:
con = sqlite3.connect("film.db")
cur = con.cursor()

# Execute your SELECT query
cur.execute("SELECT * FROM Movie")
rows = cur.fetchall()

# Print each row
for row in rows:
    print(row)

# Close the connection after you're done
con.close()

('1. Anora', 'Romance', 7.5, ' (207K)', '2h 19m')
('2. Wicked', 'Romance', 7.4, ' (170K)', '2h 40m')
('3. Babygirl', 'Romance', 5.8, ' (64K)', '1h 54m')
('4. F*** Marry Kill', 'Romance', 5.3, ' (3.2K)', '1h 37m')
('5. The Fall Guy', 'Romance', 6.8, ' (228K)', '2h 6m')
('6. The Count of Monte-Cristo', 'Romance', 7.6, ' (38K)', '2h 58m')
('7. We Live in Time', 'Romance', 7.0, ' (57K)', '1h 48m')
('8. Role Play', 'Romance', 5.5, ' (19K)', '1h 40m')
('9. It Ends with Us', 'Romance', 6.3, ' (89K)', '2h 10m')
('10. The Idea of You', 'Romance', 6.3, ' (75K)', '1h 55m')
('11. Challengers', 'Romance', 7.0, ' (159K)', '2h 11m')
('12. My Old Ass', 'Romance', 6.9, ' (42K)', '1h 29m')
('13. On Swift Horses', 'Romance', 6.0, ' (2.5K)', '1h 59m')
('14. Queer', 'Romance', 6.4, ' (24K)', '2h 17m')
('15. Fly Me to the Moon', 'Romance', 6.6, ' (52K)', '2h 12m')
('16. Love Lies Bleeding', 'Romance', 6.6, ' (59K)', '1h 44m')
('17. The Crow', 'Romance', 4.7, ' (37K)', '1h 51m')
('18. A Nice Indian Boy', 'Ro

In [67]:
import csv
import re

# Function to clean Votes
def clean_votes(vote_str):
    vote_str = vote_str.strip().replace('(', '').replace(')', '').replace(' ', '')
    if 'K' in vote_str:
        return int(float(vote_str.replace('K', '')) * 1000)
    elif 'M' in vote_str:
        return int(float(vote_str.replace('M', '')) * 1000000)
    else:
        try:
            return int(vote_str)
        except:
            return 0

# Function to clean Movie Name
def clean_movie_name(name):
    return re.sub(r'^\d+\.\s*', '', name).strip()

# Function to clean Duration ➔ total minutes (as integer)
def clean_duration(duration_str):
    hours = re.search(r'(\d+)h', duration_str)
    minutes = re.search(r'(\d+)m', duration_str)
    
    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))
    
    return total_minutes

# Read original CSV and write cleaned CSV
with open('all_genre_movies.csv', 'r', encoding='utf-8') as infile, \
     open('genre_movies.csv', 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = next(reader)
    writer.writerow(['MovieName', 'Genre', 'Rating', 'Votes', 'DurationMinutes'])  # Cleaned header

    for row in reader:
        movie, genre, rating, votes, duration = row
        movie = clean_movie_name(movie)
        votes = clean_votes(votes)
        duration_clean = clean_duration(duration)
        writer.writerow([movie, genre, float(rating), votes, duration_clean])

print("✅ CSV cleaned and saved as genre_movies.csv")


✅ CSV cleaned and saved as genre_movies.csv


In [68]:
import pandas as pd

df = pd.read_csv('genre_movies.csv')
print(df.head(10))   # Show first 10 rows


                   MovieName    Genre  Rating   Votes  DurationMinutes
0                     Wicked  Romance     7.4  174000              160
1                      Anora  Romance     7.5  211000              139
2            We Live in Time  Romance     7.0   59000              108
3                   Babygirl  Romance     5.8   65000              114
4               The Fall Guy  Romance     6.8  229000              126
5  The Count of Monte-Cristo  Romance     7.6   38000              178
6            It Ends with Us  Romance     6.3   90000              130
7                Challengers  Romance     7.0  160000              131
8            The Idea of You  Romance     6.3   76000              115
9         Fly Me to the Moon  Romance     6.6   53000              132


In [ ]:
#1#
import sqlite3
con = sqlite3.connect("films.db")
cur = con.cursor()

In [71]:
import sqlite3
import csv

# Step 1: Connect to the database
con = sqlite3.connect("films.db")
cur = con.cursor()

# Step 2: Create the table (with correct names — no spaces)
cur.execute('''
CREATE TABLE IF NOT EXISTS film (
    MovieName TEXT,
    Genre TEXT,
    Rating REAL,
    Votes INTEGER,
    Duration TEXT
)
''')

# Step 3: Open CSV and insert rows
with open('genre_movies.csv', "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row

    for row in reader:
        # row = ['Anora', 'Romance', '7.5', '170000', '2h 40m']
        cur.execute("""
            INSERT INTO film (MovieName, Genre, Rating, Votes, Duration)
            VALUES (?, ?, ?, ?, ?)
        """, row)

# Step 4: Commit and close
con.commit()
con.close()

In [62]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# Drop the table named 'film' if it exists
cursor.execute('DROP TABLE IF EXISTS film')

# Commit changes and close connection
conn.commit()
conn.close()

print("Table 'film' dropped successfully!")



Table 'film' dropped successfully!


In [79]:
import sqlite3
import pandas as pd

# Connect to database
con = sqlite3.connect('films.db')

# Query the table into a DataFrame
df = pd.read_sql_query("SELECT * FROM film", con)

# Display the first few rows
print(df.head(10))

# Close connection
con.close()


                   MovieName    Genre  Rating   Votes Duration
0                     Wicked  Romance     7.4  174000      160
1                      Anora  Romance     7.5  211000      139
2            We Live in Time  Romance     7.0   59000      108
3                   Babygirl  Romance     5.8   65000      114
4               The Fall Guy  Romance     6.8  229000      126
5  The Count of Monte-Cristo  Romance     7.6   38000      178
6            It Ends with Us  Romance     6.3   90000      130
7                Challengers  Romance     7.0  160000      131
8            The Idea of You  Romance     6.3   76000      115
9         Fly Me to the Moon  Romance     6.6   53000      132


In [73]:
import sqlite3
import pandas as pd

# Connect to the database
con = sqlite3.connect('films.db')

# Load the film table into a DataFrame
df = pd.read_sql_query("SELECT * FROM film", con)
